## Titanic kaggle competition

### Data import

In [1]:
import pandas as pd
from sklearn import preprocessing

df_train = pd.read_csv(r"C:\Users\trent\github_repos\titanic-0\train.csv")
df_test = pd.read_csv(r"C:\Users\trent\github_repos\titanic-0\test.csv")

df_train.set_index('PassengerId',inplace=True)
df_test.set_index('PassengerId',inplace=True)

df_train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Cleaning training set

In [2]:
from myFunctions import MissingData
import myFunctions as myFn
from myFunctions import ModelSelection

df = df_train.copy()

# fill in missing values
categoricalNumericFeatures = ['Pclass','Survived'] # pclass is numeric but actually is a categorical feature
missingData = MissingData(df, categoricalImputer='most_frequent',numericImputer='mean', categoricalNumericFeatures=categoricalNumericFeatures, dropColPct=.7)
df = missingData.fillMissingData()

# bin rare categorical feature values. If the column is full of rare categorical values, then drop it.
df = myFn.bin_groups(df,categoricalNumericFeatures, percent=.05)
df = myFn.drop_cols_too_unique(df,percent=.8)

# dummy code
label = 'Survived'
df = myFn.get_dummies(df,categoricalNumericFeatures, label = label)

# scaling data
dfMinMax = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df),columns=df.columns, index=df.index)
#dfMinMax.hist(figsize=(15,10))

dropped the Cabin column. Over 70% of records were Null.
imputed 2 categorical column(s) using most_frequent.
imputed 1 numeric column(s) using mean.
Missing data check complete. No columns have missing data.
dropped Name, 100.0% unique
dropped Ticket, 100.0% unique


### Model selection

In [3]:
fittedModel = ModelSelection.fit_crossvalidate_clf(dfMinMax,label,k=10,r=5,repeat=False)

Best model was Voting, with an estimated 82.38% correct test set classification rate


### Cleaning test set

In [4]:
# fill in missing values
categoricalNumericFeatures = ['Pclass'] # pclass is numeric but actually is a categorical feature
missingData = MissingData(df_test, categoricalImputer='most_frequent',numericImputer='mean', categoricalNumericFeatures=categoricalNumericFeatures, dropColPct=.7)
df_test = missingData.fillMissingData()

# bin rare categorical feature values. If the column is full of rare categorical values, then drop it.
df_test = myFn.bin_groups(df_test,categoricalNumericFeatures, percent=.05)
df_test = myFn.drop_cols_too_unique(df_test,percent=.8)

# dummy code
df_test = myFn.get_dummies(df_test,categoricalNumericFeatures)

# scaling
dfMinMax2 = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df_test),columns=df_test.columns, index=df_test.index)

dropped the Cabin column. Over 70% of records were Null.
imputed 1 categorical column(s) using most_frequent.
imputed 2 numeric column(s) using mean.
Missing data check complete. No columns have missing data.
dropped Name, 100.0% unique
dropped Ticket, 100.0% unique


### Making predictions

In [5]:
prediction = fittedModel.predict(dfMinMax2)
predictions = {'PassengerId':dfMinMax2.index.array,'Survived':prediction}
dfPredict = pd.DataFrame(predictions,dtype=int)

In [6]:
dfPredict.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [7]:
dfPredict.to_csv('predictions.csv',index=False)